# Construire un système QA avec BERT sur Wikipedia

Les systèmes de question-réponse à domaine ouvert (ODQA) se composent généralement de deux composants principaux : un récupérateur et un lecteur. Ces composants travaillent ensemble pour extraire des informations pertinentes d'un document donné et générer une réponse à la question de l'utilisateur. Voici un aperçu de leur fonctionnement :

### 1. **Récupérateur :**
   - **Objectif :** Le rôle du récupérateur est d'identifier rapidement un sous-ensemble de documents susceptibles de contenir la réponse à la question de l'utilisateur. Il s'agit souvent de ce qu'on appelle la récupération de documents.
   - **Techniques :**
      - **TF-IDF (Term Frequency-Inverse Document Frequency) :** Calcule l'importance des mots dans un document par rapport à leur fréquence dans une collection de documents.
      - **BM25 (Best Matching 25) :** Une variante de TF-IDF qui prend en compte la longueur du document et la saturation des termes.
      - **Modèles de Classement Neuronaux :** Des modèles d'apprentissage automatique, tels que ceux basés sur BERT, formés pour classer les documents en fonction de leur pertinence par rapport à une requête donnée.
   - **Sortie :** Le récupérateur fournit une liste classée de documents en fonction de leur probabilité de contenir des informations pertinentes.

### 2. **Lecteur :**
   - **Objectif :** Une fois que le récupérateur a identifié un sous-ensemble de documents, le lecteur est chargé d'extraire des passages spécifiques ou des phrases qui contiennent la réponse à la question de l'utilisateur.
   - **Techniques :**
      - **Modèles de Compréhension de Lecture Automatique (MRC) :** BERT, GPT et des modèles similaires affinés spécifiquement pour extraire des réponses de passages.
      - **Prédiction de Plage :** Le lecteur prédit les positions de début et de fin de la réponse à l'intérieur du passage sélectionné.
      - **Attention Bidirectionnelle :** Des modèles capables de prendre en compte le contexte à la fois à gauche et à droite de chaque mot, permettant une compréhension plus complète du passage.
   - **Sortie :** Le lecteur produit la réponse finale en sélectionnant le passage de texte le plus pertinent parmi les documents récupérés.

### Flux de travail de la Question-Réponse à Domaine Ouvert :

1. **Requête de l'utilisateur :** Un utilisateur saisit une question dans le système.

2. **Récupération de Documents :**
   - Le récupérateur classe les documents en fonction de leur pertinence par rapport à la question.
   - Les documents les mieux classés sont sélectionnés pour un traitement ultérieur.

3. **Extraction de Passages :**
   - Le lecteur traite les documents sélectionnés pour identifier les passages susceptibles de contenir la réponse.
   - Cela implique la tokenisation du texte et la détermination des parties les plus saillantes.

4. **Extraction de la Réponse :**
   - Le lecteur restreint davantage la recherche pour identifier le passage exact à l'intérieur des passages qui répond à la question.
   - Cela peut impliquer de prédire les positions de début et de fin de la réponse.

5. **Présentation de la Réponse :**
   - La réponse finale est présentée à l'utilisateur.

### Défis et Considérations :
- **Évolutivité :** Gérer efficacement de grandes collections de documents.
- **Taille du Modèle :** Équilibrer la taille des modèles pour la précision et l'efficacité computationnelle.
- **Raisonnement à Plusieurs Niveaux :** Répondre aux questions nécessitant des informations provenant de plusieurs documents.

Les systèmes de question-réponse à domaine ouvert ont connu des avancées significatives, en particulier avec l'utilisation de grands modèles de langage pré-entraînés, les rendant capables de comprendre et d'extraire des informations à partir de sources diverses.

In [ ]:
!pip install transformers
!pip install wikipedia==1.4.0

# Transformateurs Hugging Face
Le paquet [Hugging Face Transformers] (https://huggingface.co/transformers/#) fournit des architectures polyvalentes de pointe pour la compréhension et la génération de langage naturel. Il héberge des dizaines de modèles pré-entraînés fonctionnant dans plus de 100 langues que vous pouvez utiliser dès la sortie de la boîte. Tous ces modèles sont livrés avec une interopérabilité profonde entre PyTorch et Tensorflow 2.0, ce qui signifie que vous pouvez déplacer un modèle de TF2.0 à PyTorch et vice-versa avec seulement une ligne ou deux de code !


Si vous ne connaissez pas Hugging Face, nous vous recommandons fortement de lire le [Quickstart guide] de HF (https://huggingface.co/transformers/quickstart.html) ainsi que leurs excellents [Transformer Notebooks] (https://huggingface.co/transformers/notebooks.html) (nous l'avons fait !), car nous n'aborderons pas ce sujet dans ce cahier. Nous utiliserons [`AutoClasses`](https://huggingface.co/transformers/model_doc/auto.html), qui sert d'enveloppe autour de presque toutes les classes de base de Transformer.

## Mise au point d'un modèle de transformateur pour la réponse aux questions

Pour entraîner un Transformer pour l'AQ avec Hugging Face, nous aurons besoin de
1. choisir une architecture de modèle spécifique,
2. un ensemble de données d'AQ, et
3. le script de formation.

Avec ces trois éléments en main, nous allons ensuite parcourir le processus de réglage fin.

### 1. Choisir un modèle
Toutes les architectures Transformer ne se prêtent pas naturellement à la réponse aux questions. Par exemple, GPT ne fait pas QA ; de même, BERT ne fait pas de traduction automatique.  HF identifie les types de modèles suivants pour la tâche d'AQ :

- BERT
- distilBERT
- ALBERT
- RoBERTa
- XLNet
- XLM
- FlauBERT


Nous nous en tiendrons au modèle BERT désormais classique dans ce carnet, mais n'hésitez pas à en essayer d'autres (nous le ferons - et nous vous en informerons). Prochaine étape : un ensemble d'entraînement.


### 2. Jeu de données AQ : SQuAD
L'un des ensembles de données les plus classiques pour l'AQ est le Stanford Question Answering Dataset, ou SQuAD, qui existe en deux versions : SQuAD 1.1 et SQuAD 2.0. Ces ensembles de données de compréhension de la lecture consistent en des questions posées sur un ensemble d'articles de Wikipédia, où la réponse à chaque question est un segment (ou une portée) du passage correspondant. Dans SQuAD 1.1, toutes les questions ont une réponse dans le passage correspondant. SQuAD 2.0 augmente la difficulté en incluant des questions auxquelles le passage fourni ne permet pas de répondre.



## Utilisation d'un modèle pré-affiné provenant du dépôt Hugging Face
Si vous n'avez pas accès aux GPU ou si vous n'avez pas le temps de bricoler et d'entraîner des modèles, vous avez de la chance ! Hugging Face est bien plus qu'une collection de classes Transformer élégantes - il héberge également [un référentiel] (https://huggingface.co/models) pour les modèles pré-entraînés et affinés provenant de la vaste communauté des praticiens du NLP. Une recherche sur "squad" donne au moins 55 modèles.

![](https://github.com/fastforwardlabs/ff14_blog/blob/master/_notebooks/my_icons/HF_repo.png?raw=1)


Chacun de ces liens fournit un code explicite pour l'utilisation du modèle et, dans certains cas, des informations sur la manière dont il a été entraîné et sur les résultats obtenus. Chargeons l'un de ces modèles prédéfinis.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# l'exécution de ces commandes pour la première fois initie un téléchargement des
# poids du modèle dans ~/.cache/torch/transformers/
tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Essayons notre modèle !

Que vous ayez peaufiné votre propre modèle ou que vous ayez utilisé un modèle déjà peaufiné, il est temps de jouer avec ! QA comporte trois étapes :
1. symboliser les données d'entrée
2. obtenir les scores du modèle
3. obtenir l'étendue de la réponse

Ces étapes sont examinées en détail dans les HF [Carnets de notes des transformateurs] (https://huggingface.co/transformers/notebooks.html).

In [3]:
question = "Who ruled Macedonia"

context = """Macedonia was an ancient kingdom on the periphery of Archaic and Classical Greece,
and later the dominant state of Hellenistic Greece. The kingdom was founded and initially ruled
by the Argead dynasty, followed by the Antipatrid and Antigonid dynasties. Home to the ancient
Macedonians, it originated on the northeastern part of the Greek peninsula. Before the 4th
century BC, it was a small kingdom outside of the area dominated by the city-states of Athens,
Sparta and Thebes, and briefly subordinate to Achaemenid Persia."""


# 1. SYMBOLISER L'ENTRÉE
# note : si vous n'incluez pas return_tensors='pt' vous obtiendrez une liste de listes ce qui est plus facile pour
# l'exploration, mais vous ne pouvez pas l'introduire dans un modèle.
inputs = tokenizer.encode_plus(question, context, return_tensors="pt")

# 2. OBTENIR LES SCORES DU MODÈLE
# La classe AutoModelForQuestionAnswering inclut un prédicteur d'empan au-dessus du modèle.
# Le modèle renvoie les scores de début et de fin de réponse pour chaque mot du texte.
outputs = model(**inputs)
answer_start = torch.argmax(outputs.start_logits)  # obtenir le début de réponse le plus probable avec l'argmax du score
answer_end = torch.argmax(outputs.end_logits) + 1  # obtenir la fin de réponse la plus probable avec l'argmax du score

# 3. OBTENIR LA RÉPONSE SPAN
# une fois que nous avons les tokens de début et de fin les plus probables, nous prenons tous les tokens entre eux
# et convertissons les tokens en mots !
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'the Argead dynasty'

# AQ sur les pages Wikipédia
Nous avons testé notre modèle sur une question associée à un court passage, mais que se passe-t-il si nous voulons extraire une réponse d'un document plus long ? Une page Wikipédia typique est beaucoup plus longue que l'exemple ci-dessus, et nous devons faire quelques manipulations avant de pouvoir utiliser notre modèle dans des contextes plus longs.

Commençons par afficher une page Wikipédia.

In [4]:
import wikipedia as wiki
import pprint as pp

question = 'What is the wingspan of an albatross?'

results = wiki.search(question)
print("Résultats de la recherche Wikipedia pour notre question:\n")
pp.pprint(results)

page = wiki.page(results[1])
text = page.content
print(f"\n L'article {results[1]}  de Wikipedia contient {len(text)} caractères.")

Résultats de la recherche Wikipedia pour notre question:

['Wandering albatross',
 'Black-browed albatross',
 'List of largest birds',
 'Argentavis',
 'Pelagornis',
 'List of birds by flight speed',
 'Mollymawk',
 'Largest body part',
 'Largest and heaviest animals',
 'Orders of magnitude (length)']

 L'article Black-browed albatross  de Wikipedia contient 16588 caractères.


In [5]:
inputs = tokenizer.encode_plus(question, text, return_tensors='pt')
print(f"Cela se traduit par {len(inputs['input_ids'][0])} tokens.")

Token indices sequence length is longer than the specified maximum sequence length for this model (4914 > 512). Running this sequence through the model will result in indexing errors


Cela se traduit par 4914 tokens.


Le tokéniseur prend l'entrée sous forme de texte et renvoie des tokens. En général, les tokenizers convertissent des mots ou des morceaux de mots dans un format compatible avec le modèle. Les tokens et le format spécifiques dépendent du type de modèle. Par exemple, BERT génère les mots différemment de RoBERTa. Veillez donc à toujours utiliser le tokéniseur associé qui convient à votre modèle.

Dans ce cas, le tokenizer convertit notre texte d'entrée en 8824 tokens, mais cela dépasse de loin le nombre maximum de tokens qui peuvent être introduits dans le modèle en une seule fois. La plupart des modèles de type BERT ne peuvent accepter que 512 tokens à la fois, d'où l'avertissement (quelque peu déroutant) ci-dessus (comment se fait-il que 10 > 512 ?). Cela signifie que nous devrons diviser notre entrée en morceaux et que chaque morceau ne doit pas dépasser 512 tokens au total.

Lorsque l'on travaille avec la réponse aux questions, il est essentiel que chaque morceau suive le format suivant :

[CLS] jetons de question [SEP] jetons de contexte [SEP]

Cela signifie que, pour chaque segment d'un article de Wikipédia, nous devons faire précéder la question d'origine, suivie du "chunk" suivant de tokens d'article.

In [6]:
# time to chunk !
from collections import OrderedDict
# identifier les jetons de la question (token_type_ids = 0)
qmask = inputs['token_type_ids'].lt(1)
qt = torch.masked_select(inputs['input_ids'], qmask)
print(f"La question consiste en {qt.size()[0]} jetons.")

chunk_size = model.config.max_position_embeddings - qt.size()[0] - 1 # the "-1" accounts for
# avoir à ajouter un jeton [SEP] à la fin de chaque morceau
print(f"Chaque morceau contiendra {chunk_size - 2} jetons de l'article de Wikipédia.")

# créer un dict de dicts ; chaque sous-dict imite la structure de l'entrée du modèle pré-chunked
chunked_input = OrderedDict()
for k,v in inputs.items():
    q = torch.masked_select(v, qmask)
    c = torch.masked_select(v, ~qmask)
    chunks = torch.split(c, chunk_size)

    for i, chunk in enumerate(chunks):
        if i not in chunked_input:
            chunked_input[i] = {}

        thing = torch.cat((q, chunk))
        if i != len(chunks)-1:
            if k == 'input_ids':
                thing = torch.cat((thing, torch.tensor([102])))
            else:
                thing = torch.cat((thing, torch.tensor([1])))

        chunked_input[i][k] = torch.unsqueeze(thing, dim=0)

La question consiste en 12 jetons.
Chaque morceau contiendra 497 jetons de l'article de Wikipédia.


In [7]:
for i in range(len(chunked_input.keys())):
    print(f"Nombre de jetons dans le bloc{i}: {len(chunked_input[i]['input_ids'].tolist()[0])}")

Nombre de jetons dans le bloc0: 512
Nombre de jetons dans le bloc1: 512
Nombre de jetons dans le bloc2: 512
Nombre de jetons dans le bloc3: 512
Nombre de jetons dans le bloc4: 512
Nombre de jetons dans le bloc5: 512
Nombre de jetons dans le bloc6: 512
Nombre de jetons dans le bloc7: 512
Nombre de jetons dans le bloc8: 512
Nombre de jetons dans le bloc9: 423


Each of these chunks (except for the last one) has the following structure:

[CLS], 12 question tokens, [SEP], 497 tokens of the Wikipedia article, [SEP] token = 512 tokens

Each of these chunks can now be fed to the model without causing indexing errors. We'll get an "answer" for each chunk; however, not all answers are useful, since not every segment of a Wikipedia article is informative for our question. The model will return the [CLS] token when it determines that the context does not contain an answer to the question.

In [8]:
def convert_ids_to_string(tokenizer, input_ids):
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids))

answer = ''

# maintenant nous itérons sur nos chunks, en cherchant la meilleure réponse pour chaque chunk
for _, chunk in chunked_input.items():
    outputs = model(**chunk)

    answer_start = torch.argmax(outputs.start_logits)  # obtenir le début de réponse le plus probable avec l'argmax du score
    answer_end = torch.argmax(outputs.end_logits) + 1  # obtenir la fin de réponse la plus probable avec l'argmax du score


    ans = convert_ids_to_string(tokenizer, chunk['input_ids'][0][answer_start:answer_end])
    print(ans)
    # Si la réponse == [CLS], le modèle n'a pas trouvé de réponse réelle dans ce morceau.
    if ans != '[CLS]':
        answer += ans + " / "

print(answer)

[CLS]
200 to 240 cm ( 79 – 94 in )
[CLS]
[CLS]
[CLS]
[CLS]
[CLS]
[CLS]
[CLS]
[CLS]
200 to 240 cm ( 79 – 94 in ) / 
